In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Google Drive hesabınıza bağlanın

import os
import pandas as pd
import re
import unicodedata

folder_path = "/content/drive/MyDrive/data"  # Dosyaların yüklendiği klasörün yolu

txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]  # .txt uzantılı dosyaların listesi

data = pd.DataFrame()  # Boş bir veri çerçevesi oluşturun

for file in txt_files:
    file_path = os.path.join(folder_path, file)  # Dosya yolunu oluşturun
    with open(file_path, 'r', encoding='cp1252') as f:  # Dosyayı okuyun
        content = f.read()  # Dosyanın içeriğini okuyun
        title, text = content.split('\n\n', 1)  # İçeriği başlık ve metin olarak ayır
        data = data.append({'file_name': file, 'title': title, 'text': text}, ignore_index=True)  # Veri çerçevesine dosya adı, başlık ve metni ekle

fake_data = pd.read_csv('/content/drive/MyDrive/data/fakenewdata.csv')
data = pd.concat([data, fake_data[['title', 'text']]], ignore_index=True)  # Veri çerçevesine 'fakenewdata.csv' dosyasındaki başlık ve metni ekle

def prepare_text(text):
    # Girdi string değilse string'e dönüştür
    if not isinstance(text, str):
        text = str(text)

    # Noktalama işaretlerini kaldır
    text = re.sub(r'[^\w\s]', '', text)

    # Rakamları kaldır
    text = re.sub(r'\d+', '', text)

    # Büyük harfleri küçük harfe çevir
    text = text.lower()

    # Latin alfabesi dışındaki bütün karakterleri kaldır
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # İngilizce karakterler dışındaki bütün karakterleri kaldır
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # re.sub fonksiyonu None döndürürse, string'in kendisini geri döndür
    return text if text is not None and re.sub(r'[^\\w\\s]', '', text) is not None else ''


# Başlık ve metin sütunlarındaki verileri önişlemden geçir
data['title'] = data['title'].apply(prepare_text)
data['text'] = data['text'].apply(prepare_text)
data = data.head(10000)


print(data)

Mounted at /content/drive


<ipython-input-6-a60d2c315674>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'file_name': file, 'title': title, 'text': text}, ignore_index=True)  # Veri çerçevesine dosya adı, başlık ve metni ekle
<ipython-input-6-a60d2c315674>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'file_name': file, 'title': title, 'text': text}, ignore_index=True)  # Veri çerçevesine dosya adı, başlık ve metni ekle
<ipython-input-6-a60d2c315674>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'file_name': file, 'title': title, 'text': text}, ignore_index=True)  # Veri çerçevesine dosya adı, başlık ve metni ekle
<ipython-input-6-a60d2c315674>:20: FutureWarning: The frame.append 

               file_name                                              title  \
0     politics (308).txt                  talks aim to avert pension strike   
1     politics (354).txt                   cabinet anger at brown cash raid   
2     politics (329).txt                    milburn defends poster campaign   
3     politics (349).txt                    blair joins school sailing trip   
4     politics (346).txt                   chancellor rallies labour voters   
...                  ...                                                ...   
9995                 NaN     tillerson says iraq must resist iran influence   
9996                 NaN  ai weiwei and warhol together again  the new y...   
9997                 NaN  while you were watching the world series some ...   
9998                 NaN  saleh was killed in rpg gun attack on his car ...   
9999                 NaN  olympic ban adds to russias culture of grievan...   

                                                   

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping


# Verileri train ve test setlerine ayıran fonksiyon
def split_data(data, test_size=0.2):
    train, test = train_test_split(data, test_size=test_size, random_state=42)
    return train, test

# Train ve test setlerine ayır
train_data, test_data = split_data(data)

# Eğitim verilerinin özellikleri ve etiketleri
train_X = train_data['text'].astype(str).values
train_Y = train_data['title'].astype(str).values

# Test verilerinin özellikleri ve etiketleri
test_X = test_data['text'].astype(str).values
test_Y = test_data['title'].astype(str).values


# Verileri LSTM modeline uygun hale getir
def prepare_data(X, Y, max_len):
    chars = sorted(list(set(''.join(X))))
    char_to_int = dict((c, i) for i, c in enumerate(chars))

    X_vec = []
    for d in X:
        X_vec.append([char_to_int[char] for char in d])
    X_vec = pad_sequences(X_vec, maxlen=max_len, padding='post')
    X_vec = np.reshape(X_vec, (X_vec.shape[0], X_vec.shape[1], 1))

    Y_vec = []
    for d in Y:
        Y_vec.append([char_to_int[char] for char in d])
    Y_vec = pad_sequences(Y_vec, maxlen=max_len, padding='post')
    Y_vec = np.array(Y_vec)

    return X_vec, Y_vec, chars

max_len = 7088
train_X, train_Y, chars = prepare_data(train_X, train_Y, max_len)
test_X, test_Y, chars = prepare_data(test_X, test_Y, max_len)


# LSTM modeli oluştur
model = Sequential()
model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(train_Y.shape[1], activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)
model.fit(train_X, train_Y, epochs=10, batch_size=64, validation_data=(test_X, test_Y), callbacks=[early_stop])
model.save('model.h5')

In [36]:
import numpy as np
from tensorflow.keras.models import load_model

# Modeli yükle
model = load_model('model.h5')

# Tek bir girdiye dayalı olarak tahmin yap
def generate_title(text, chars, max_len):
    char_to_int = dict((c, i) for i, c in enumerate(chars))

    X_vec = []
    for d in text:
        X_vec.append([char_to_int[char] for char in d])
    X_vec = pad_sequences(X_vec, maxlen=max_len, padding='post')
    X_vec = np.reshape(X_vec, (X_vec.shape[0], X_vec.shape[1], 1))

    prediction = model.predict(X_vec)
    title = ''
    for p in prediction[0]:
        title += chars[np.argmax(p)]
    print(title)


# Örnek girdi
text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed auctor magna quis nisl imperdiet, in rhoncus enim viverra. In vulputate tincidunt metus, vitae rhoncus orci hendrerit vel. Sed sed sagittis metus. Proin finibus mauris vel ex tincidunt, eu consequat mi convallis. Maecenas quis ultrices ante. In sit amet nunc mollis, bibendum nibh eu, gravida augue. Vestibulum euismod elit ut metus facilisis, ac consequat est molestie. Sed nec turpis eget massa gravida dictum.'
text = prepare_text(text)
max_len = 7088

# Başlık oluştur
generate_title(text, chars, max_len)



15/15 [==============================] - 70s 5s/step
																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																			